In [1]:
import numpy as np
import os; os.chdir("..")

In [4]:
from benchmarks.simple_QP.load_simple_QP import (
    SimpleQPDataset,
    create_dataloaders,
    dc3_dataloader,
)

dataset_dir = "/home/aterpin/projects/hcnn/src/benchmarks/simple_QP/datasets"

def load_data(
    use_DC3_dataset,
    use_convex,
    problem_seed,
    problem_var,
    problem_nineq,
    problem_neq,
    problem_examples,
):
    """Load problem data."""
    if not use_DC3_dataset:
        # Choose problem parameters
        if use_convex:
            filename = (
                f"SimpleQP_seed{problem_seed}_var{problem_var}_ineq{problem_nineq}"
                f"_eq{problem_neq}_examples{problem_examples}.npz"
            )
        else:
            raise NotImplementedError()
        dataset_path = os.path.join(dataset_dir, filename)

        QPDataset = SimpleQPDataset(dataset_path)
        train_loader, valid_loader, test_loader = create_dataloaders(
            dataset_path, batch_size=2048, val_split=0.1, test_split=0.1
        )
        Q, p, A, G, h = QPDataset.const
        p = p[0, :, :]
        X = QPDataset.X
    else:
        # Choose the filename here
        if use_convex:
            filename = (
                f"dc3_random_simple_dataset_var{problem_var}_ineq{problem_nineq}"
                f"_eq{problem_neq}_ex{problem_examples}"
            )
        else:
            filename = (
                f"dc3_random_nonconvex_dataset_var{problem_var}_ineq{problem_nineq}"
                f"_eq{problem_neq}_ex{problem_examples}"
            )
        filename_train = filename + "train.npz"
        dataset_path_train = os.path.join(dataset_dir, filename_train)
        filename_valid = filename + "valid.npz"
        dataset_path_valid = os.path.join(dataset_dir, filename_valid)
        filename_test = filename + "test.npz"
        dataset_path_test = os.path.join(dataset_dir, filename_test)
        train_loader = dc3_dataloader(dataset_path_train, use_convex, batch_size=2048)
        valid_loader = dc3_dataloader(
            dataset_path_valid, use_convex, batch_size=1024, shuffle=False
        )
        test_loader = dc3_dataloader(
            dataset_path_test, use_convex, batch_size=1024, shuffle=False
        )
        Q, p, A, G, h = train_loader.dataset.const
        p = p[0, :, :]
        X = train_loader.dataset.X

    return (filename, Q, p, A, G, h, X, train_loader, valid_loader, test_loader)

In [ ]:
# 1. Load a batch of data for autotuning
filename, Q, p, A, G, h, X, train_loader, valid_loader, test_loader = load_data(
    use_DC3_dataset=True,
    use_convex=True,
    problem_seed=42,
    problem_var=100,
    problem_nineq=50,
    problem_neq=50,
    problem_examples=10000,
)
X_batch, _ = next(iter(train_loader))

In [ ]:
# 2. Sigma grid
sigma_candidates = np.arange(0.0, 10.05, 0.05)
sigma_candidates[0] += 0.01
print(sigma_candidates)

# 3. Candidate training iterations
candidate_iterations = np.concatenate([
    np.arange(10, 100, 10), np.arange(100, 500, 50), np.arange(500, 1000, 100)
])
print(candidate_iterations)

# 4. Define the number of n_iter_backward for given n_iter_train
def get_n_iter_backward(n_iter_train):
    if n_iter_train <= 25:
        return n_iter_train
    elif n_iter_train <= 250:
        return 25
    return 25 + (n_iter_train - 250) // 10

[ 0.01  0.05  0.1   0.15  0.2   0.25  0.3   0.35  0.4   0.45  0.5   0.55
  0.6   0.65  0.7   0.75  0.8   0.85  0.9   0.95  1.    1.05  1.1   1.15
  1.2   1.25  1.3   1.35  1.4   1.45  1.5   1.55  1.6   1.65  1.7   1.75
  1.8   1.85  1.9   1.95  2.    2.05  2.1   2.15  2.2   2.25  2.3   2.35
  2.4   2.45  2.5   2.55  2.6   2.65  2.7   2.75  2.8   2.85  2.9   2.95
  3.    3.05  3.1   3.15  3.2   3.25  3.3   3.35  3.4   3.45  3.5   3.55
  3.6   3.65  3.7   3.75  3.8   3.85  3.9   3.95  4.    4.05  4.1   4.15
  4.2   4.25  4.3   4.35  4.4   4.45  4.5   4.55  4.6   4.65  4.7   4.75
  4.8   4.85  4.9   4.95  5.    5.05  5.1   5.15  5.2   5.25  5.3   5.35
  5.4   5.45  5.5   5.55  5.6   5.65  5.7   5.75  5.8   5.85  5.9   5.95
  6.    6.05  6.1   6.15  6.2   6.25  6.3   6.35  6.4   6.45  6.5   6.55
  6.6   6.65  6.7   6.75  6.8   6.85  6.9   6.95  7.    7.05  7.1   7.15
  7.2   7.25  7.3   7.35  7.4   7.45  7.5   7.55  7.6   7.65  7.7   7.75
  7.8   7.85  7.9   7.95  8.    8.05  8.1   8.15  8

In [10]:
import signal
class GracefulShutdown:
    """A context manager for graceful shutdowns."""

    stop = False

    def __enter__(self):
        """Register the signal handler."""

        def handle_signal(signum, frame):
            self.stop = True

        signal.signal(signal.SIGINT, handle_signal)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Unregister the signal handler."""
        pass

In [ ]:
def evaluate_params(sigma, n_iter_train):
    n_iter_bwd = get_n_iter_backward(n_iter_train)
    return 1.0, 0.4

In [ ]:
import time
time_budget_s = 10
target_cv = 0.001

t_start = time.time()

best_accuracy = 0.0
best_params = (1.0, 100)
with GracefulShutdown() as gs:
    while time.time() - t_start < time_budget_s and not gs.stop:
        for sigma in sigma_candidates:
            for n_iter_train in candidate_iterations:
                accuracy, cv = evaluate_params(sigma, n_iter_train)
                if cv < target_cv and accuracy < best_accuracy:
                    print(f"New best found: ({sigma}, {n_iter_train}).")
                    print(f" -> {accuracy:.4f} (cv: {cv:.4f})")
                    best_accuracy = accuracy
                    best_params = (sigma, n_iter_train)

print(best_params, best_accuracy)